# SQL query from table names - Continued

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

# Load path from the environment variable
env_ih1 = os.getenv("ENV_IH1")

dotenv_path = Path(env_ih1)
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "name",
      "type": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE TABLE Employees (
             ID, int, primary key,
             name, str             
             )

 CREATE TABLE Salary(
             UUID, str, primary key,
             ID_employee, int
             Year, date,
             Salary, float        
            )

             
 CREATE TABLE Studies(
             UUID, str, primary key,
             ID_employee, int,             
             Graduation year, date,
             Educational level, int,
             Specialty, str,
             Instituion, str
            )
             
"""} ]



In [4]:
print(context)

[{'role': 'system', 'content': '\n CREATE TABLE Employees (\n             ID, int, primary key,\n             name, str             \n             )\n\n CREATE TABLE Salary(\n             UUID, str, primary key,\n             ID_employee, int\n             Year, date,\n             Salary, float        \n            )\n\n             \n CREATE TABLE Studies(\n             UUID, str, primary key,\n             ID_employee, int,             \n             Graduation year, date,\n             Educational level, int,\n             Specialty, str,\n             Instituion, str\n            )\n             \n'}]


In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key= OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("""The name of the best paid employee""", context_user))

```sql
SELECT e.name
FROM Employees e
JOIN Salary s ON e.ID = s.ID_employee
ORDER BY s.Salary DESC
LIMIT 1;
```


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("The name of the best paid employee", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the name of the employee who is the best paid by joining the "employees" table with the "salary" table on the employee ID. It then orders the result by salary in descending order and limits the output to only the top result, which represents the best paid employee.


In [9]:
#new
print(return_CCRMSQL("Return the Institution with the higher average salary", context_user))

```sql
SELECT Institution
FROM Studies
JOIN Employees ON Studies.ID_employee = Employees.ID
JOIN Salary ON Employees.ID = Salary.ID_employee
GROUP BY Institution
ORDER BY AVG(Salary) DESC
LIMIT 1;
```


In [10]:
#old
print(return_CCRMSQL("Return the Institution with the higher average salary", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(s.salary) AS avg_salary
FROM salary s
JOIN employees e ON s.ID_usr = e.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the `salary` table with the `employees` table on the `ID_usr` column. It calculates the average salary for each institution by grouping the results by `Institution`. Then, it orders the results by the average salary in descending order and limits the output to only the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?